### Leer el archivo JSON usando el "DataFrameReader" de Spark

### Ingestion de la carpeta "movie_language"

In [0]:
dbutils.widgets.text("p_enviroment", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
movies_language_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("languageId", IntegerType(), True),
    StructField("languageRoleId", IntegerType(), True)

])

In [0]:
movies_language_df = spark.read.schema(movies_language_schema).option("multiLine", True).json(
    f"{bronze_folder_path}/{v_file_date}/movie_language/"
)

In [0]:
from pyspark.sql.functions import current_timestamp, lit


In [0]:
movies_language_with_columns_df = movies_language_df.withColumnsRenamed({
    'movieId': 'movie_id',
    'languageId': 'language_id',
}).withColumns({
    'ingestion_date': current_timestamp(),
    'enviroment': lit("production"),
    'file_date': lit(v_file_date)
})

In [0]:
movies_languages_final_df = movies_language_with_columns_df.drop("languageRoleId")

In [0]:
#spark.sql("DROP TABLE IF EXISTS movie_silver.movies_languages")

overwrite_partition(movies_languages_final_df, "movie_silver", "movies_languages", "file_date")


In [0]:
#movies_languages_final_df.write.mode("overwrite").parquet("/mnt/datamoviehistory/silver/movies_languages")

#movies_languages_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.movies_languages")

movies_languages_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movies_languages")


In [0]:
%fs
ls /mnt/datamoviehistory/silver/movies_casts

In [0]:
%sql
select file_date, count(1) from movie_silver.movies_languages group by file_date;